**Лабораторная работа 3**

ИУ5И-21М Хаммуд Хала

**Цель работы**

Изучение продвинутых способов предварительной обработки данных для дальнейшего формирования моделей.

**Задание**

Выбрать один или несколько наборов данных (датасетов) для решения следующих задач. Каждая задача может быть решена на отдельном датасете, или несколько задач могут быть решены на одном датасете. Просьба не использовать датасет, на котором данная задача решалась в лекции.

Для выбранного датасета (датасетов) на основе материалов лекций решить следующие задачи:

масштабирование признаков (не менее чем тремя способами);

обработку выбросов для числовых признаков (по одному способу для удаления выбросов и для замены выбросов);

обработку по крайней мере одного нестандартного признака (который не является числовым или категориальным);

отбор признаков:

один метод из группы методов фильтрации (filter methods);

один метод из группы методов обертывания (wrapper methods);

один метод из группы методов вложений (embedded methods).

**Ход работы**

In [4]:
from google.colab import files
uploaded = files.upload()

Saving amazon_sales_data 2025.csv to amazon_sales_data 2025.csv


In [5]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
import pandas as pd

# Загрузка данных Amazon
df = pd.read_csv("amazon_sales_data 2025.csv")

# Выбор числовых признаков
numeric_features = ['Price', 'Quantity', 'Total Sales']

# Инициализация разных масштабаторов
scaler_standard = StandardScaler()
scaler_minmax = MinMaxScaler()
scaler_robust = RobustScaler()

# Применяем масштабирование к копиям датасета
df_standard = df.copy()
df_standard[numeric_features] = scaler_standard.fit_transform(df_standard[numeric_features])

df_minmax = df.copy()
df_minmax[numeric_features] = scaler_minmax.fit_transform(df_minmax[numeric_features])

df_robust = df.copy()
df_robust[numeric_features] = scaler_robust.fit_transform(df_robust[numeric_features])

# Просмотр результатов (пример)
df_standard[numeric_features].head()


,Price,Quantity,Total Sales
0,-0.746511,0.100937,-0.636505
1,-0.641213,0.801892,-0.460449
2,-0.746511,-0.600017,-0.684520
3,-0.746511,0.100937,-0.636505
4,-0.509590,0.100937,-0.420437


Масштабирование признаков (3 способа)

In [7]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
import pandas as pd

# Загрузка данных
df = pd.read_csv("amazon_sales_data 2025.csv")  # Укажите корректное имя файла

# Определим числовые признаки, которые будем масштабировать
columns_to_scale = ['Price', 'Quantity']

# 1. Стандартизация (StandardScaler)
scaler = StandardScaler()
data_scaled1 = df.copy()
data_scaled1[columns_to_scale] = scaler.fit_transform(df[columns_to_scale])

# 2. Масштабирование Min-Max (в диапазон [0, 1])
scaler = MinMaxScaler()
data_scaled2 = df.copy()
data_scaled2[columns_to_scale] = scaler.fit_transform(df[columns_to_scale])

# 3. Robust Scaling (устойчивый к выбросам)
scaler = RobustScaler()
data_scaled3 = df.copy()
data_scaled3[columns_to_scale] = scaler.fit_transform(df[columns_to_scale])

# Просмотр результатов
print("Стандартизация:")
print(data_scaled1[columns_to_scale].head())

print("\nMin-Max Scaling:")
print(data_scaled2[columns_to_scale].head())

print("\nRobust Scaling:")
print(data_scaled3[columns_to_scale].head())


Стандартизация:
      Price  Quantity
0 -0.746511  0.100937
1 -0.641213  0.801892
2 -0.746511 -0.600017
3 -0.746511  0.100937
4 -0.509590  0.100937

Min-Max Scaling:
      Price  Quantity
0  0.037975      0.50
1  0.071730      0.75
2  0.037975      0.25
3  0.037975      0.50
4  0.113924      0.50

Robust Scaling:
      Price  Quantity
0 -0.160714       0.0
1 -0.089286       0.5
2 -0.160714      -0.5
3 -0.160714       0.0
4  0.000000       0.0


Обработка выбросов методом Тьюки

In [9]:
import pandas as pd

# Указываем путь к файлу
file_path = 'amazon_sales_data 2025.csv'

# Загружаем файл с кодировкой ISO-8859-1 (если возникают проблемы с utf-8)
df = pd.read_csv(file_path, encoding='ISO-8859-1', delimiter=',')

# Просмотр первых строк
df.head()


,Order ID,Date,Product,Category,Price,Quantity,Total Sales,Customer Name,Customer Location,Payment Method,Status
0,ORD0001,14-03-25,Running Shoes,Footwear,60,3,180,Emma Clark,New York,Debit Card,Cancelled
1,ORD0002,20-03-25,Headphones,Electronics,100,4,400,Emily Johnson,San Francisco,Debit Card,Pending
2,ORD0003,15-02-25,Running Shoes,Footwear,60,2,120,John Doe,Denver,Amazon Pay,Cancelled
3,ORD0004,19-02-25,Running Shoes,Footwear,60,3,180,Olivia Wilson,Dallas,Credit Card,Pending
4,ORD0005,10-03-25,Smartwatch,Electronics,150,3,450,Emma Clark,New York,Debit Card,Pending


In [10]:
import pandas as pd
from scipy.stats.mstats import winsorize

# Функция для удаления выбросов методом Тьюки (Tukey IQR)
def remove_outliers(df, col):
    Q1 = df[col].quantile(0.25)  # Первый квартиль
    Q3 = df[col].quantile(0.75)  # Третий квартиль
    IQR = Q3 - Q1                # Интерквартильный размах
    return df[(df[col] >= Q1 - 1.5 * IQR) & (df[col] <= Q3 + 1.5 * IQR)]

# Применяем удаление выбросов к признаку 'Total Sales'
df_clean = remove_outliers(df, 'Total Sales')

# Замена выбросов методом винзоризации (Winsorization)
df['Total_Sales_winsorized'] = winsorize(df['Total Sales'], limits=[0.05, 0.05])

# Просмотр результата
df[['Total Sales', 'Total_Sales_winsorized']].head()


,Total Sales,Total_Sales_winsorized
0,180,180
1,400,400
2,120,120
3,180,180
4,450,450


Обработка нестандартного признака

In [12]:
# Указываем путь к файлу
file_path = 'amazon_sales_data 2025.csv'

# Загрузка данных
df = pd.read_csv(file_path, encoding='ISO-8859-1')

# Просмотр уникальных значений нестандартного признака — например, "Product"
print(df['Product'].unique())


['Running Shoes' 'Headphones' 'Smartwatch' 'T-Shirt' 'Smartphone' 'Book'
 'Jeans' 'Laptop' 'Washing Machine' 'Refrigerator']


In [27]:
# Оставим только строки с одной запятой
df_valid = df[df['Customer Location'].str.count(',') == 1].copy()

# Разделим колонку 'Customer Location' на две — 'City' и 'Country'
split_cols = df_valid['Customer Location'].str.split(',', expand=True)

# Убедимся, что результат действительно содержит 2 столбца
if split_cols.shape[1] == 2:
    split_cols.columns = ['City', 'Country']

    # Удалим пробелы
    split_cols['City'] = split_cols['City'].str.strip()
    split_cols['Country'] = split_cols['Country'].str.strip()

    # Объединим с оригиналом
    df_valid = pd.concat([df_valid, split_cols], axis=1)

    # Просмотр результата
    print(df_valid[['Customer Location', 'City', 'Country']].head())
else:
    print("❌ Ошибка: некоторые строки содержат больше или меньше одной запятой.")


❌ Ошибка: некоторые строки содержат больше или меньше одной запятой.


In [14]:
# Убедимся, что столбец 'Date' имеет строковой тип
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')

# Создаём новые признаки на основе даты
df['Year'] = df['Date'].dt.year      # Год
df['Month'] = df['Date'].dt.month    # Месяц
df['Day'] = df['Date'].dt.day        # День

# Проверяем результат
df[['Date', 'Year', 'Month', 'Day']].head()


<ipython-input-14-919ec028f1be>:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Date'] = pd.to_datetime(df['Date'], errors='coerce')


,Date,Year,Month,Day
0,2025-03-14,2025,3,14
1,2025-03-20,2025,3,20
2,2025-02-15,2025,2,15
3,2025-02-19,2025,2,19
4,2025-10-03,2025,10,3


Отбор признаков

In [30]:
# Импорт необходимых библиотек
import pandas as pd
from sklearn.feature_selection import (SelectKBest, f_classif,
                                       RFE, SelectFromModel)
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

# Загрузка данных
df = pd.read_csv('amazon_sales_data 2025.csv')

# Предобработка: удалим строки с пропущенными значениями (если есть)
df.dropna(inplace=True)

# Пример: выберем признаки и целевой признак (Sales)
# Убедимся, что признаки числовые
X = df[['Price', 'Quantity']]  # Признаки
y = df['Total Sales']          # Целевая переменная (например, для регрессии или классификации)

# Преобразуем целевую переменную в категории, если нужна классификация
# (например, высокий / низкий уровень продаж)
y_class = pd.qcut(y, q=2, labels=[0, 1])  # 0 = низкие продажи, 1 = высокие продажи

# --- 1. Отбор признаков с помощью SelectKBest ---
print("SelectKBest (f_classif):")
selector_kbest = SelectKBest(score_func=f_classif, k='all')
X_kbest = selector_kbest.fit_transform(X, y_class)
print(pd.DataFrame({
    'Feature': X.columns,
    'Score': selector_kbest.scores_
}))

# --- 2. Рекурсивное исключение признаков (RFE) ---
print("\nRFE (с SVC):")
svc = SVC(kernel='linear')
rfe = RFE(estimator=svc, n_features_to_select=1)
rfe.fit(X, y_class)
print(pd.DataFrame({
    'Feature': X.columns,
    'Ranking': rfe.ranking_
}))

# 3. Отбор признаков с помощью модели RandomForestClassifier
print("\nSelectFromModel (RandomForestClassifier):")

model = RandomForestClassifier(n_estimators=100, random_state=0)

# Обучаем модель перед использованием важностей
model.fit(X, y_class)

sfm = SelectFromModel(model)
sfm.fit(X, y_class)

# Выводим важность признаков
print(pd.DataFrame({
    'Feature': X.columns,
    'Importance': model.feature_importances_
}))



SelectKBest (f_classif):
    Feature       Score
0     Price  367.710867
1  Quantity    6.471634

RFE (с SVC):
    Feature  Ranking
0     Price        2
1  Quantity        1

SelectFromModel (RandomForestClassifier):
    Feature  Importance
0     Price    0.913369
1  Quantity    0.086631


In [31]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import SelectKBest, f_classif, RFE, SelectFromModel
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

# 1. Загрузка данных
df = pd.read_csv("/content/amazon_sales_data 2025.csv")

# 2. Подготовка признаков и целевой переменной
# Предположим, что целевой признак — это 'Category'
X = df[['Price', 'Quantity', 'Total Sales']]  # عددية فقط
y = df['Category']  # الهدف: نوع المنتج

# 3. Кодируем y, если это строки
le = LabelEncoder()
y_encoded = le.fit_transform(y)

# 4. Filter Method: ANOVA
selector = SelectKBest(score_func=f_classif, k='all')  # نختار все ونطبع الأعلى
X_new = selector.fit_transform(X, y_encoded)
print("Выбранные признаки (ANOVA):", X.columns[selector.get_support()])
print("ANOVA оценки:")
for col, score in zip(X.columns, selector.scores_):
    print(f"{col}: {score:.2f}")

# 5. Wrapper Method: RFE
estimator = SVC(kernel="linear")
selector_rfe = RFE(estimator, n_features_to_select=2)
X_rfe = selector_rfe.fit_transform(X, y_encoded)
print("Выбранные признаки (RFE):", X.columns[selector_rfe.get_support()])

# 6. Embedded Method: Random Forest
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X, y_encoded)
selector_emb = SelectFromModel(model, threshold='median', prefit=True)
X_emb = selector_emb.transform(X)
print("Выбранные признаки (Random Forest):", X.columns[selector_emb.get_support()])


Выбранные признаки (ANOVA): Index(['Price', 'Quantity', 'Total Sales'], dtype='object')
ANOVA оценки:
Price: 123.60
Quantity: 0.33
Total Sales: 53.46
Выбранные признаки (RFE): Index(['Price', 'Quantity'], dtype='object')
Выбранные признаки (Random Forest): Index(['Price', 'Total Sales'], dtype='object')


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
